In [1]:
import pylangacq as pla

In [2]:
pla.__version__

'0.11.0'

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
import glob

In [6]:
import os

In [7]:
import pickle

In [8]:
import torch

In [9]:
import nltk

import re
from collections import Counter

import string

import pprint

In [10]:
import math

Data grab

In [11]:
# pd.options.display.max_seq_items = 2000

pd.set_option('display.max_columns', None)

In [13]:
#ignore header for easier (ordinal) columnar indexing
datasheet = pd.read_excel('CogData_FU_82818.xlsx',header=None)

In [99]:
# column headers
# 2 - memory responses
# 3 - tech responses
datasheet.iloc[0]

0                                             Record ID
1                                            Event Name
2     Write a short sketch about a memory from your ...
3     How does technology and social media impact th...
4                   Date of Neurobehavioral Status Exam
                            ...                        
92                         Cog State: One Back Accuracy
93                          Ravens Progressive Matrices
94                                Logical Memory Part B
95                         East Boston Immediate Recall
96                                            Complete?
Name: 0, Length: 97, dtype: object

Row Indices of Memory Responses 

In [14]:
#ignore first (header) column
memory_ix = datasheet.iloc[1:][datasheet.iloc[1:,2].notnull()].index

Record IDs of Memory Respondents [Analyze all scores from each participant]

In [15]:
memory_id = list(datasheet.iloc[memory_ix][0])

Tech respondents

In [16]:
tech_ix = datasheet.iloc[1:][datasheet.iloc[1:,3].notnull()].index

In [18]:
tech_id = list(datasheet.iloc[tech_ix][0])

In [19]:
#each respondent responded to one prompt
len(tech_id) == len(set(tech_id))

True

In [21]:
# 74th col lists the participant's mmse 
datasheet.iloc[0][74]

'Total MMSE score'

In [20]:
ix_score_latest_completed = []
for ix in tech_id:
    slice_ix = list(datasheet[datasheet[0]==ix].index)
#     while datasheet.iloc[max(slice_ix)][96] != 'Complete':
    # if there is no mmse score, take most recent score
    while np.isnan(datasheet.iloc[max(slice_ix)][74]):
        slice_ix.remove(max(slice_ix))
    ix_score_latest_completed += [max(slice_ix)]

In [21]:
mem_ix_score_latest_completed = [i for i in ix_score_latest_completed if datasheet.iloc[i,0] in memory_id]

In [22]:
tech_mmse = pd.Series([y for y in datasheet.iloc[ix_score_latest_completed,74]])

In [126]:
# ix by samples 
tech_mmse_dict = {ix:score for (ix,score) in zip(tech_ix,tech_mmse)}

In [127]:
# torch.save(tech_mmse_dict,'tech_mmse_dict.pt')

In [23]:
mem_mmse = pd.Series([y for y in datasheet.iloc[mem_ix_score_latest_completed,74]])

In [24]:
mem_mmse_dict = {ix:score for (ix,score) in zip(memory_ix,mem_mmse)}

In [125]:
# torch.save(mem_mmse_dict,'mem_mmse_dict.pt')

Load addb vocab glove word embeddings 

In [25]:
import torch

In [26]:
glove_path = './PretrainedWordEmb/'

### Helper functions - tokenization

In [27]:
# returns nested list

def list_tokenize(document):
    sentences = nltk.sent_tokenize(document)
    return [nltk.word_tokenize(sent) for sent in sentences]


In [31]:
list_tokenize(datasheet.iloc[memory_ix[6],2])

[['I',
  'have',
  'great',
  'memories',
  'of',
  'growing',
  'up',
  'in',
  'huge',
  'house',
  'with',
  'a',
  'large',
  'garden',
  'full',
  'of',
  'flowers',
  'and',
  'fruit',
  'trees',
  '.'],
 ['I',
  'specifically',
  'remember',
  'enjoying',
  'climbing',
  'the',
  'trees',
  'to',
  'pick',
  'plums',
  ',',
  'pears',
  'and',
  'peaches',
  'and',
  'maybe',
  'even',
  'apples',
  '.'],
 ['The',
  'lower',
  'portion',
  'of',
  'the',
  'garden',
  'was',
  'reserved',
  'for',
  'vegetables',
  'and',
  'I',
  'was',
  "n't",
  'too',
  'interested',
  'in',
  'that',
  '.'],
 ['This',
  'is',
  'one',
  'of',
  'my',
  'best',
  'childhood',
  'memories',
  'of',
  'being',
  'outdoors',
  '.']]

In [32]:
#https://www.nltk.org/book/ch07.html
def get_pos(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    pos_list = [p for sent in sentences for (t,p) in sent]
    return pos_list

In [33]:
get_pos(datasheet.iloc[memory_ix[6],2])

['PRP',
 'VBP',
 'JJ',
 'NNS',
 'IN',
 'VBG',
 'RP',
 'IN',
 'JJ',
 'NN',
 'IN',
 'DT',
 'JJ',
 'NN',
 'JJ',
 'IN',
 'NNS',
 'CC',
 'NN',
 'NNS',
 '.',
 'PRP',
 'RB',
 'VBP',
 'VBG',
 'VBG',
 'DT',
 'NNS',
 'TO',
 'VB',
 'NNS',
 ',',
 'NNS',
 'CC',
 'NNS',
 'CC',
 'RB',
 'RB',
 'NNS',
 '.',
 'DT',
 'JJR',
 'NN',
 'IN',
 'DT',
 'NN',
 'VBD',
 'VBN',
 'IN',
 'NNS',
 'CC',
 'PRP',
 'VBD',
 'RB',
 'RB',
 'JJ',
 'IN',
 'DT',
 '.',
 'DT',
 'VBZ',
 'CD',
 'IN',
 'PRP$',
 'JJS',
 'NN',
 'NNS',
 'IN',
 'VBG',
 'NNS',
 '.']

---------------------------------------------

### Grab word embeddings for APC Vocabulary  

In [37]:
vocabset = set()

In [38]:
for ix in tech_ix:
    for sent in list_tokenize(datasheet.iloc[ix,3]):
        vocabset.update(sent)

In [39]:
for ix in memory_ix:
    for sent in list_tokenize(datasheet.iloc[ix,2]):
        vocabset.update(sent)

In [40]:
len(vocabset)

2125

Map APC POS (nltk): ADDB POS (transcript)

In [41]:
posset = set()

In [42]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [44]:
addb_posset = torch.load('pos_dict.pt')

In [46]:
for ix in memory_ix:
    posset.update(get_pos(datasheet.iloc[ix,2]))

In [47]:
for ix in tech_ix:
    posset.update(get_pos(datasheet.iloc[ix,3]))

In [48]:
posset.difference(addb_posset)

{'$',
 "''",
 '(',
 ')',
 ',',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'TO',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WRB'}

In [49]:
#swap with chat term where poss
posdict = {k:k for k in posset}

In [ ]:
# References for mapping
# https://talkbank.org/manuals/MOR.html#_Toc17967934
# https://talkbank.org/manuals/MOR.html#_Toc1313898
# nltk.help.upenn_tagset()

In [223]:
# nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [52]:
posdict['CC'] = 'CONJ'

posdict['CD'] = 'DET:NUM'

posdict['DT'] = 'DET:ART'

posdict['EX'] = 'PRO:EXIST'

posdict['IN'] = 'PREP'

posdict['JJ'] = 'ADJ'

posdict['JJR'] = 'ADJ'

posdict['JJS'] = 'ADJ'

posdict['MD'] = 'MOD'

posdict['NN'] = 'N'

posdict['NNP'] = 'N:PROP'

posdict['NNPS'] = 'N:PROP'

posdict['NNS'] = 'N'

posdict['PDT'] = 'DET:DEM'

posdict['POS'] = 'DET:POSS'

posdict['PRP'] = 'PRO:PER'

posdict['PRP$'] = 'PRO:POSS'

posdict['RB'] = 'ADV'

posdict['RBR'] = 'ADV'

posdict['RBS'] = 'ADV'

posdict['RP'] = 'PART'

posdict['TO'] = 'PREP'

posdict['VB'] = 'V'

posdict['VBD'] = 'V'

posdict['VBG'] = 'PART'

posdict['VBN']  = 'PART'

posdict['VBP'] = 'V'

posdict['VBZ'] = 'V'

# DET:INT not in addb_posseet
posdict['WDT'] = 'DET:POSS'

posdict['WP'] = 'PRO:INT'

# ADV:WH not in addb_posset
posdict['WRB'] = 'ADV'

In [53]:
[missing for missing in posset.difference(set(addb_posset.keys())) if posdict[missing] not in set(addb_posset.keys())]

[')', '(', ',', '$', ':', 'FW', "''"]

In [304]:
# torch.save(posdict,'apc_pos_dict.pt')

In [54]:
# posdict = torch.load('apc_pos_dict.pt')

--------------------------------------------------------------------------------------------------

VOCAB EMB

In [54]:
#https://github.com/Blosc/bcolz
import bcolz

In [57]:
# import zipfile

In [55]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [32]:
# with zipfile.ZipFile('./PretrainedWordEmb/glove.42B.300d.zip', 'r') as zip_ref:
#     zip_ref.extractall('./PretrainedWordEmb/')

load pretrained glove (common crawl 42B tokens, 1.9M vocab, uncased, 300d vectors)

In [56]:
glove_path = './PretrainedWordEmb/'

In [52]:
# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         break
    

In [58]:
# words = []
# idx = 0
# word2idx = {}
# vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')

# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)

In [30]:
# vectors = bcolz.carray(vectors[1:].reshape((-1,300)), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')
# vectors.flush()

In [31]:
# pickle.dump(words, open(f'{glove_path}/glove.42B.300_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'{glove_path}/glove.42B.300_idx.pkl', 'wb'))

In [57]:
vectors = bcolz.open(f'{glove_path}/glove.42B.300.dat')[:]

In [58]:
words = pickle.load(open(f'{glove_path}/glove.42B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/glove.42B.300_idx.pkl', 'rb'))

In [59]:
# UNK token -- average of embeddings
average_vec = np.mean(vectors, axis=0)
# print(average_vec)

In [60]:
'UNK' in words

False

In [61]:
type(words)

list

In [62]:
type(word2idx)

dict

In [63]:
# replace unknown word with average embedding
words.append('UNK')

In [64]:
vectors.shape

(1917494, 300)

In [65]:
vectors = np.concatenate((vectors,np.reshape(average_vec,(1,300))))

In [66]:
len(vectors)

1917495

In [67]:
word2idx['UNK'] = len(vectors) - 1

In [68]:
len(words)

1917495

In [69]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer

def get_glovembs(sample):
    
    def handle_contractions(x):
        tokenizer = TreebankWordTokenizer()
        x = tokenizer.tokenize(x)
        x = ' '.join(x)
        return x
    
    # preprocess acc to pretrained embeddings 
    # lower
    # replace _, -
    # rm contractions
    # rm nonascii chars
    
    sample = [s for s in sample]
    
    preprocessample = [handle_contractions(t).split()[0] for t in [token.lower().replace('_','-') for token in sample]]
    
    
    preprocessample = [''.join([i if ord(i) < 128 else '' for i in w]) for w in preprocessample]
    
    preprocessample = [t.translate(str.maketrans('', '', string.punctuation)) for t in preprocessample]
    
# uncomment for traceunks = get_glovembs(vocabset)
#     return set([word for word in preprocessample if word not in words])
    # {w: vectors[word2idx[w]] for w in words}
    return {sample[i]: torch.tensor(vectors[word2idx[preprocessample[i]]]) if preprocessample[i] in words else torch.tensor(vectors[word2idx['UNK']]) for i in range(len(sample))}

In [111]:
traceunks = get_glovembs(vocabset)

In [95]:
traceunks = set()
for word in vocabset:
    if word not in words:
        traceunks.add(word)

In [112]:
traceunks

{'',
 '11alpha',
 '2ndteam',
 'devicelogs',
 'dontuse',
 'electroniceven',
 'envirmental',
 'etccwas',
 'gerslyn',
 'intigral',
 'monmarte',
 'precomputer',
 'preinternet',
 'townno',
 'unqua',
 'windowdoors',
 'wlodawski'}

In [97]:
len(traceunks)

302

In [70]:
trace_vocab_embs = get_glovembs(vocabset)

In [71]:
len(trace_vocab_embs)

2125

In [116]:
# pickle.dump(trace_vocab_embs, open(f'{glove_path}/trace.vocab.glove.42B.300_words.pkl', 'wb'))